In [26]:
import os
from dotenv import load_dotenv

In [27]:
load_dotenv()

True

In [28]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [29]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [31]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
import langchain

#Get the prompt for the using for ChatOpenAI
prompt = langchain.hub.pull("hwchase17/openai-functions-agent")

#Choose the LLM
llm = ChatOpenAI(model="gpt-4-turbo-preview")
#llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

#Construct the OpenAI Functions Agent
agent_rumble = create_openai_functions_agent(llm=llm, prompt=prompt, tools=tools)

In [32]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent_runnable=agent_rumble, tools= tools)

In [34]:
agent_executor.invoke(
    {"input":"Who is the winner of the US Open", "chat_history":[]}
)

{'input': 'Who is the winner of the US Open',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': "The winner of the US Open in 2023 is Novak Djokovic. He won the men's singles final by defeating Daniil Medvedev with scores of 6-3, 7-6(5), 6-3. This victory marked Djokovic's 24th Grand Slam singles title, tying Margaret Court's record and bolstering his case to be considered one of the greatest tennis players of all time."}, log="The winner of the US Open in 2023 is Novak Djokovic. He won the men's singles final by defeating Daniil Medvedev with scores of 6-3, 7-6(5), 6-3. This victory marked Djokovic's 24th Grand Slam singles title, tying Margaret Court's record and bolstering his case to be considered one of the greatest tennis players of all time."),
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'US Open 2023 winner'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'US Open 2023 winner'}`

In [35]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple, Annotated, TypedDict
from collections import deque
import operator

class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [36]:
from langchain_core.pydantic_v1 import BaseModel

class Plan(BaseModel):
    """Plan to follow the step in the future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [37]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_template(
    """
    For the given objective, come up with a simple step by step plan. \
        This plan should involve the individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
        The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip the steps.

        {objective}
    """
)

planner = create_structured_output_runnable(
    Plan,
    ChatOpenAI(model = "gpt-4-turbo-preview", temperature=0),
    planner_prompt
)

In [38]:
planner.invoke({
    "objective": "What is the hometown of the current Australia Open Winner?"
})

Plan(steps=['Identify the current year to determine the most recent Australia Open tournament.', 'Find out the winner of the most recent Australia Open in the identified year.', 'Research the hometown of the identified winner.'])

In [39]:
from langchain.chains.openai_functions import create_openai_fn_runnable
from langchain_core.pydantic_v1 import BaseModel

class Response(BaseModel):
    """Response back to User."""
    response: str

replanner_prompt = ChatPromptTemplate.from_template(
    """
For the given objective, come up with a simple step by step plan.\
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip the steps.

Your Objective was this: 
{input}

Your Original Plan was this: 
{plan}

You have currently done the follow steps: 
{past_steps}

Update the plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan. 
    """
)

replanner = create_openai_fn_runnable(
    [Plan, Response],
    ChatOpenAI(model = "gpt-4-turbo-preview", temperature=0),
    replanner_prompt,
)

In [40]:
async def execute_step(state: PlanExecute):
    task = state["plan"][0]
    agent_response = await agent_executor.ainvoke({"input": task, "chat_history": []})
    return {
        "past_steps": (task, agent_response["agent_outcome"].return_values["output"]),
    }

async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"objective": state["input"]})
    return {"plan": plan.steps}

async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output, Response):
        return {
            "response": output.response
        }
    else: 
        return {
            "plan": output.steps
        }
    
def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return True
    else:
        return False
    

In [41]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(PlanExecute)

#Add the plan node
workflow.add_node("planner", plan_step)

#Add the executor_step
workflow.add_node("agent", execute_step)

#Add the Replan_Node
workflow.add_node("replan", replan_step)

workflow.set_entry_point("planner")

#From plan we will go to the agent
workflow.add_edge("planner", "agent")

#From agent we will go to the replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    #Next, we pass in the function that will determine which node is called next
    should_end,
    {
        #If `tools` then we call the tool node.
        True: END,
        False: "agent",
    },
)

app = workflow.compile()

In [46]:

from langchain_core.messages import HumanMessage

config = {"recursion_limit": 50}
inputs = {"input": "Can you Code main.tf for Terraform for automating the instances for the AWS EC2?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():  
        for m, n in v.items():
            if m=="response":
                print(f"Final Response: {n}")

Final Response: All necessary steps to automate the creation of AWS EC2 instances using Terraform have been completed, including verification of the instance creation. No further steps are needed.
